## Installation and deployment of gNB and FlexRIC of Open Air Interface (OAI)
## Objectives
- Install and configure the gNB and FlexRIC 
- Deploy a gNB together a previous configured 5GC and connect to FlexRIC
- Inspec KPM and RC xApps

## FlexRIC Installation

To install OAI FlexRIC, go back to /home/dco1020 and follow this tutorial.

**Step 0:** go back to /home/dco1020. In a Linux Terminal, type:

sudo su

cd /home/dco1020

**Step 1:** Install dependencies:

sudo apt-get install -y autotools-dev automake g++ make libpcre2-dev byacc cmake python3-dev libsctp-dev pcre2-utils bison

**Step 2:** Install swig:

git clone https://github.com/swig/swig.git

cd swig

git checkout release-4.1 

./autogen.sh

./configure --prefix=/usr/

make -j8

sudo make install

**Step 3:** Clone FlexRIC repository:

git clone https://gitlab.eurecom.fr/mosaic5g/flexric flexric

cd flexric/

git checkout 29af8408afad3da36824bd7912562202023ae924

**Step 4:** Build FlexRIC:

mkdir build && cd build && cmake .. && make -j8

sudo make install

## gNB Installation

To install OAI RAN 5G, go back to /home/dco1020 and follow this tutorial.

**Step 0:** go back to /home/dco1020. In a Linux Terminal, type:

sudo su

cd /home/dco1020


**Step 1:** Install dependencies

sudo add-apt-repository ppa:ettusresearch/uhd

sudo apt-get install libuhd-dev uhd-host -y

sudo apt-get install libuhd4.6.0 -y

sudo apt-get install ccache -y

sudo apt-get install net-tools

**Step 2:** Fix problem in the libuhd. When you run the commands below, if you get the error "cannot access archive", you can continue to the next step.
sudo dpkg -i --force-overwrite /var/cache/apt/archives/libuhd4.6.0_4.6.0.0-0ubuntu1~focal1_amd64.deb

**Step 3:** Clone OpenAirInterface 5G RAN repository

git clone https://gitlab.eurecom.fr/oai/openairinterface5g.git

cd /home/dco1020/openairinterface5g

git checkout v2.1.0

**Step 4:** Run the installation script

source oaienv

cd cmake_targets

sudo ./build_oai -I 

sudo ./build_oai -I --install-optional-packages

sudo ./build_oai -w USRP --ninja --gNB --nrUE --build-e2 -C --build-lib "uescope nrscope telnetsrv" 

**Attention**: you may find Wireshark configuration screen, select "yes" to continue.

**Attention**: be patient, the last command could take a whileeeee. If you want to follow the installation logs, run the command below:

tail -f /home/dco1020/openairinterface5g/cmake_targets/log/all.txt

If you have any compilation error, please, try:

Take care about conflits between our VM and some windows configuration, If you have compilation problem, please, check and disable WSL, hipervison and virtual machine platforms, acoording to the following figures.


![gNB_problem1](../FIGS/H02_5GRAN_UNI_III/gNB_problem1.png)

![gNB_problem2](../FIGS/H02_5GRAN_UNI_III/gNB_problem2.png)

## gNB Configuration for the monolithic mode

In a Linux Terminal, follow the steps below:

**Step 1:** Configure the PLMN and TAC values

The Public Land Mobile Network (PLMN) and Tracking Area Code (TAC) will be configured in the monolithic RAN (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf) configuration file. 

Figure below shows the part of the code that must be modified in the monolithic file (lines 17 and 18). The parameters shown in figure should be modified according to the values present in docker compose in the "oai-amf" session (/home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml). 

![gNB_config_01](../FIGS/H02_5GRAN_UNI_III/gNB_config_01.png)


Figure below shows the parameters that should be checked and passed to the monolithic configuration file: PLMN_SUPPORT_TAC (referring to "tracking_area_code"), PLMN_SUPPORT_MCC (referring to "mcc"), PLMN_SUPPORT_MNC (referring to "mnc"), SST_0 (referring to "sst") and the SD_0 (referring to "sd"). If the "sd" parameter is not in the monolithic configuration file, simply add it according to line 18 in previous figure.

![gNB_config_02](../FIGS/H02_5GRAN_UNI_III/gNB_config_02.png)


Open files by typing:

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf

gedit /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml

**Step 2:**  Configure the network interfaces

This part you will configure the N2 interface in (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf), such as in figure below. This interface is responsible to connect the RAN and the AMF.

![gNB_config_03](../FIGS/H02_5GRAN_UNI_III/gNB_config_03.png)

You need to verify/change the IP addresses. You can find the OAI-AMF IP address inside /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml, such as in figure below. You must get this IP address and put it in the "amf_ip_address" parameter, in the "ipv4" field (such as in previous figure).

![gNB_config_04](../FIGS/H02_5GRAN_UNI_III/gNB_config_04.png)


Check if the IP Address of gnb.sa.band78.fr1.106PRB.usrpb210.conf is the same of docker-compose-basic-nrf.yaml. To edit gnb.sa.band78.fr1.106PRB.usrpb210.conf, type:

gedit /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf


**Step 3:** To check a virtual interface (demo-oai), type:

ifconfig -a

You should find a virtual interface (demo-oai) (as shown in figure below). This interface serves to connect the gNodeB with OAI-AMF.  You must get its IP address and name to configure the N2 interface such as two figures before. 

![gNB_config_05](../FIGS/H02_5GRAN_UNI_III/gNB_config_05.png)

## gNB Connection Configuration to FlexRIC 

In a Linux Terminal, follow the steps below:

**Step 1:** To configure gNodeB to connect to FlexRIC, add the code from the image below to the end of the gNodeB file (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf)

![gNB_config_06](../FIGS/H06_FlexRIC_KPM_RC/gnb_flexric_configuration.png)


## Deployment for FlexRIC

In a Linux Terminal, follow the step below:

**Step 1:** Run FlexRIC:

cd flexric/

./build/examples/ric/nearRT-RIC

The expected return is shown in figure below:

![flexric_01](../FIGS/H06_FlexRIC_KPM_RC/expected_result_flexric.png)

## Deployment for the monolithic RAN 

The deployment of the monolithic RAN  without dashboard, but if you want to deploy the OAI gNB dashboard follow this tutorial: [OAI gNB dashboard](https://github.com/vicentesousa/PPGEEC_2333/blob/main/howtos/H02_5G_OAI_dashboard.ipynb).

**Attention:** You must have Core 5G up and running

**Step 1:** open a new terminal using ctrl+Alt+T and type:

sudo su

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx 


**Step 2**: To check the connection of gNodeB to the Core, see AMF logs. Open another terminal (the third one) using ctrl+Alt+T and type: 

docker logs -f oai-amf

The expected return is shown in figure below, where gNodeB shows the connected status.

![gNB_config_06](../FIGS/H02_5GRAN_UNI_III/gNB_config_06.png)

**Attention:** In addition to viewing the gNodeB connection on the Core 5G terminal, you need to check the FlexRIC terminal, which should detect the gNodeB, as shown in the image below:

![flexric_02](../FIGS/H06_FlexRIC_KPM_RC/gnb_detected_in_flexric.png
)

## Deployment for UE

There are some important parameters in the UE deployment command. First, in "RFSIMULATOR" the IP 192.168.70.129 is the first valid IP of the subnet configured in the installation of the gNB. The parameters "-r", "--numerology", "--band" and "-C" are respectively the number of PRBs, the 5G numerology, the operating band and the frequency of the synchronization block (SSB).  

The deployment below is without the OAI UE dashboard, but if you want to deploy the dashboard, follow this tutorial: [OAI UE dashboard](https://github.com/vicentesousa/PPGEEC_2333/blob/main/howtos/H02_5G_OAI_dashboard.ipynb).

**Step 1:** to deploy a UE, type in a terminal:

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf

**Step 2**: Expected results:

To verify that the connection with the UE has been established, make sure that a tunnel for the UE has been created. In a terminal, type:

ifconfig --all

The expected result is shown in figure below, where an IP address has been assigned to the UE.


![UE_config_05](../FIGS/H03_5GUE_UNI_III/UE_config_05.png)



## Run KPM/RC xApps 